In [26]:
import numpy as np
import pandas as pd
import os

computer_path= os.getcwd()

# path_input_data = computer_path + os.path.sep+ r"data"
# path_EPCs = computer_path + os.path.sep+ r"data" + os.path.sep + r"EPCs"
# saving_path = computer_path


path_input_data = os.getcwd()+os.path.sep+"Data"
path_EPCs = r"D:\OneDrive - Cardiff University\04 - Projects\03 - PhD\03 - Analysis\03 - LSOAs\00 - Data\EPC and energy efficiency\all-domestic-certificates" #computer_path + os.path.sep + r"Data" +os.path.sep +r"EPCs"
saving_path = r"D:\OneDrive - Cardiff University\04 - Projects\03 - PhD\03 - Analysis\03 - LSOAs\00 - Data\Domestic_EPC_results\EPC_costs" #computer_path

### Functions used to extract the number of dwelling type and heating system from each EPC

In [27]:
def getEPC(path, file):
    file = 'certificates.csv'
    df = pd.read_csv(path+os.path.sep+file)
    df["POSTCODE"] = df["POSTCODE"].str.replace(' ','')
    df=pd.merge(df, OA_lookup_df.loc[:,["PCD7", "LSOA11CD", "MSOA11CD", "LAD11CD"]],left_on="POSTCODE", right_on="PCD7", how='left')

    df.drop(["PCD7"], axis=1, inplace=True)
    df.replace('Bungalow', 'House', regex=True, inplace=True)
    df.replace('Maisonette', 'Flat', regex=True, inplace=True)
    df["Heat consumption (kWh)"]=0
    df["Space constrained ratio"]=0
    df["Heat_rating_changes [%]"]=0
    df["Heat_efficiency_changes [%]"]=0
    print(df.shape)
    return df
    
def setFlag(df):
    #Remove rows without a valid LSOA, MSOA or LA
    df["FLAG LSOA"] = df["LSOA11CD"].astype(str).str.len()==9
    df.drop(df[df["FLAG LSOA"]==False].index, inplace=True)

    df["FLAG MSOA"] = df["MSOA11CD"].astype(str).str.len()==9
    df.drop(df[df["FLAG MSOA"]==False].index, inplace=True)

    df["FLAG LA"] = df["LAD11CD"].astype(str).str.len()==9
    df.drop(df[df["FLAG LA"]==False].index, inplace=True)
    
    return df

def costTokWh(df):
    print("list of fuels: ", df["MAIN_FUEL"].unique()) 
    cost_column = "HEATING_COST_CURRENT" 
    #USING SAP price from january 2016: https://www.bre.co.uk/filelibrary/SAP/2012/RdSAP-fuel-prices-from-January-2018.xlsx
    price_of_elec = 15.32/100 #df_fuels.loc["Electricity", "Cost [GBP/kWh]"]
    price_of_gas = 4.32/100 #df_fuels.loc["Ngas", "Cost [GBP/kWh]"] 
    price_of_oil = 5.06/100 #df_fuels.loc["Oil", "Cost [GBP/kWh]"] 
    price_of_solid_fuel = 5.67/100 #df_fuels.loc["Biomass", "Cost [GBP/kWh]"] 

    #Efficiency from "combined"  individual_tech database
    eff_elec = 1
    eff_gas = 0.84
    eff_oil = 0.84
    eff_solid_fuel = 0.82
    eff_hp = 3.4
    
    df["Heat system"] = None
    df["MAINHEAT_DESCRIPTION"].fillna("No value", inplace=True)
    df["Heat consumption (kWh)"].dropna(inplace=True)
    
    gas_list = ["mains gas (not community)",
                #'mains gas - this is for backwards compatibility only and should not be used'
                ]
    
    df.loc[df["MAIN_FUEL"].isin(gas_list), "Heat consumption (kWh)"] = df.loc[df["MAIN_FUEL"].isin(gas_list), cost_column]/price_of_gas*eff_gas
    df.loc[df["MAIN_FUEL"].isin(gas_list), "Heat system"] = "Gas boiler"

    electricity_list = ["electricity (not community)", 'electricity - this is for backwards compatibility only and should not be used',
                       'Electricity: electricity, unspecified tariff']
    
    df.loc[df["MAIN_FUEL"].isin(electricity_list), "Heat system"]='Resistance heating'
    df.loc[df["MAINHEAT_DESCRIPTION"].str.contains("heat pump"),"Heat system"]="Heat pump"
    
    df.loc[df["Heat system"]=='Resistance heating', "Heat consumption (kWh)"] = df.loc[df["Heat system"]=='Resistance heating', cost_column]/price_of_elec*eff_elec
    df.loc[df["Heat system"]=="Heat pump", "Heat consumption (kWh)"] = df.loc[df["Heat system"]=="Heat pump", cost_column]/price_of_elec*eff_hp

    oil_list = ['appliances able to use mineral oil or liquid biofuel','oil (not community)', 'oil - this is for backwards compatibility only and should not be used']
    
    df.loc[df["MAIN_FUEL"].isin(oil_list), "Heat system"] = "Oil boiler"
    df.loc[df["Heat system"]=="Oil boiler", "Heat consumption (kWh)"] = df.loc[df["Heat system"]=="Oil boiler", cost_column]/price_of_oil*eff_oil

    solid_fuels = ['anthracite', 'house coal (not community)','wood logs', 'smokeless coal', 'house coal - this is for backwards compatibility only and should not be used',
                  'bulk wood pellets', 'wood chips']

    df.loc[df["MAIN_FUEL"].isin(solid_fuels), "Heat system"]="Solid fuel boiler"
    df.loc[df["Heat system"]=="Solid fuel boiler", "Heat consumption (kWh)"] = df.loc[df["Heat system"]=="Solid fuel boiler", cost_column]/price_of_solid_fuel*eff_solid_fuel
    return df

def threshold(df):
    # Remove inaccurate value from the dataset. e.g. everything under 25 kWh/m2
    min_kWh_m2 = 15 # minimum intensity Passive House (Passivhaus) standard is at 15 kWh/m2
    max_kWh_m2 = 400 
    max_m2 = 1000
    min_number_room = 1
    
    df["Heat per floor area (kWh/m2)"]=df["Heat consumption (kWh)"]/df["TOTAL_FLOOR_AREA"]
    print(df.loc[(df["Heat per floor area (kWh/m2)"]<min_kWh_m2),:].shape)
    df.fillna(0, inplace=True)
    to_drop = df.loc[(df["Heat per floor area (kWh/m2)"]<min_kWh_m2),:].index
    df.drop(to_drop, inplace=True)
    print(df.loc[(df["Heat per floor area (kWh/m2)"]>max_kWh_m2),:].shape)
    to_drop = df.loc[(df["Heat per floor area (kWh/m2)"]>max_kWh_m2),:].index
    df.drop(to_drop, inplace=True)
       
    
    # Used to assess the sapce availability to change heating system 
    # Source: Element Energy, “Analysis on abating direct emissions from ‘hard-to-decarbonise’ homes, with a view to informing the UK’s long term targets,” 2019.
    df["Space constrained ratio"] = df["TOTAL_FLOOR_AREA"]/df["NUMBER_HEATED_ROOMS"]
    df.loc[df["Space constrained ratio"]<=16, "Space constrained ratio"]=1
    df.loc[df["Space constrained ratio"]>16, "Space constrained ratio"]=0
    
    # Change in EPC and impact on heat
    to_drop = df.loc[df["POTENTIAL_ENERGY_EFFICIENCY"]<df["CURRENT_ENERGY_EFFICIENCY"], :].index
    df.drop(to_drop, inplace=True)
    
    to_drop = df.loc[df["HEATING_COST_CURRENT"]<df["HEATING_COST_POTENTIAL"]*(1+0.1), :].index
    df.drop(to_drop, inplace=True)
    
    df["Heat_rating_changes [%]"] = (df["HEATING_COST_POTENTIAL"] - df["HEATING_COST_CURRENT"])/df["HEATING_COST_CURRENT"]
    df["Heat_efficiency_changes [%]"] = (df["HEATING_COST_POTENTIAL"] - df["HEATING_COST_CURRENT"])/df["HEATING_COST_CURRENT"]/(df["POTENTIAL_ENERGY_EFFICIENCY"] - df["CURRENT_ENERGY_EFFICIENCY"])

    df["Heat_rating_changes [%]"].replace([np.inf, -np.inf], 0, inplace=True)
    df["Heat_efficiency_changes [%]"].replace([np.inf, -np.inf], 0, inplace=True)
    #Remove houses with less than 1 heated room
    to_drop = df.loc[(df["NUMBER_HEATED_ROOMS"]<min_number_room),:].index
    df.drop(to_drop, inplace=True)
    
    return df


def removeDuplicates(df):
    # Keep only the most uptodate EPC
    
    df = df.sort_values(["BUILDING_REFERENCE_NUMBER", "INSPECTION_DATE"])
    df.drop_duplicates(["BUILDING_REFERENCE_NUMBER"], keep="last", inplace=True)
    
    return df

def dwellingType(df):
    
    df["Dwelling type"]=None
    df.loc[df["PROPERTY_TYPE"]=="Flat", "Dwelling type"]="Flat"
    df.loc[df["PROPERTY_TYPE"]=="House", "Dwelling type"]=df.loc[df["PROPERTY_TYPE"]=="House", "BUILT_FORM"]
    list_dwelling_type = list(df["Dwelling type"].unique())
    list_dwelling_type = list(filter(None.__ne__, list_dwelling_type)) # Remove None from the list
    # temp_df.replace({dwel:"Terrace" for dwel in list_dwelling_type if "Terrace" in dwel}, regex=True, inplace=True)
   
    to_replace = {dwel:"Terraced" for dwel in list_dwelling_type if "Terrace" in dwel}
    df.replace(to_replace, inplace=True)
    df.replace("Semi-Detached", "Semi-detached", inplace=True)
    list_dwelling_type = list(df["Dwelling type"].unique())
    print(list_dwelling_type)
    
    to_drop = df.loc[(df["Dwelling type"]=='NO DATA!'),:].index
    df.drop(to_drop, inplace=True)
    
    df["Key"] = df["Dwelling type"].str.lower()+" "+df["Heat system"].str.lower()
    df["Key_EPC"] = df["CURRENT_ENERGY_RATING"]+" to "+df["POTENTIAL_ENERGY_RATING"]
    df.dropna(subset=['Key'], inplace=True)
    print("Shape:", df.shape)
    
    
    return df



def populatingSkeleton(df, skeleton, target):
    list_col = list(skeleton.columns)
    key = "Key"
    if target=="Heat consumption (kWh)":
        list_col = [x for x in list_col if "Mean" in x]
        prefix = "Mean"
    elif target=="NUMBER_HEATED_ROOMS":
        list_col = [x for x in list_col if "rooms" in x]
        prefix = "Average number of rooms of"
    elif target=="TOTAL_FLOOR_AREA":
        list_col = [x for x in list_col if "floor" in x]
        prefix = "Average floor area of"
    elif target=="Space constrained ratio":
        list_col = [x for x in list_col if "space constrained" in x]
        prefix = "Average space constrained"
    elif target=="Heat_rating_changes [%]":
        
        list_col = [x for x in list_col if "EPC rating" in x]
        prefix = "EPC rating"
        key= "Key_EPC"
    elif target=="Heat_efficiency_changes [%]":
        list_col = [x for x in list_col if "EPC efficiency" in x]
        prefix = "EPC efficiency"
        key= "Key_EPC"
        
    print(list_col, prefix)
    df[key] = [prefix+" "+x for x in df[key].values]
    print(df[key].unique())
    
    df = df.loc[df[key].isin(list_col), :].copy()
    # Add the value available at LSOA level
    LSOA_df_count = df.groupby(["LAD11CD", "MSOA11CD", "LSOA11CD", key]).count()[target]
    LSOA_df_count[LSOA_df_count<4]=np.nan #Remove the archetypes with too few properties
    LSOA_df = df.groupby(["LAD11CD", "MSOA11CD", "LSOA11CD", key]).sum()[target]
    LSOA_df = LSOA_df/LSOA_df_count
    LSOA_df = LSOA_df.to_frame()
    LSOA_df = LSOA_df.unstack(3)
    LSOA_df.columns = LSOA_df.columns.droplevel(0)
    
    populated_skeleton = pd.merge(skeleton, LSOA_df, left_index=True, right_index=True, how='left')
    for col in list_col:
        mergeColumn(populated_skeleton, col)
    populated_skeleton.reset_index(level=2, inplace=True)
    
    # Add the value available at MSOA level
    MSOA_df_count = df.groupby(["LAD11CD", "MSOA11CD", key]).count()[target]
    MSOA_df_count[MSOA_df_count<4]=np.nan #Remove the archetypes with too few properties
    MSOA_df = df.groupby(["LAD11CD", "MSOA11CD", key]).sum()[target]
    MSOA_df = MSOA_df/MSOA_df_count
    MSOA_df = MSOA_df.to_frame()
    MSOA_df = MSOA_df.unstack(2)
    MSOA_df.columns = MSOA_df.columns.droplevel(0)

    populated_skeleton = pd.merge(populated_skeleton, MSOA_df, left_index=True, right_index=True, how='left')
    for col in list_col:
        mergeColumn(populated_skeleton, col)
        
    populated_skeleton.reset_index(level=1, inplace=True)
    
    # Add the value available at LA level
    LA_df_count = df.groupby(["LAD11CD", key]).count()[target]
    LA_df_count[LA_df_count<4]=np.nan #Remove the archetypes with too few properties
    LA_df = df.groupby(["LAD11CD", key]).sum()[target]
    LA_df = LA_df/LA_df_count
    LA_df = LA_df.to_frame()
    LA_df = LA_df.unstack(1)
    LA_df.columns = LA_df.columns.droplevel(0)
    LA_df.fillna(LA_df.mean() ,inplace=True)
    
    populated_skeleton = pd.merge(populated_skeleton, LA_df, left_index=True, right_index=True, how='left')
    for col in list_col:
        mergeColumn(populated_skeleton, col)
        
    populated_skeleton.reset_index(inplace=True)
    populated_skeleton.set_index(["LAD11CD", "MSOA11CD", "LSOA11CD"], inplace=True, drop=True)
    
    #populated_skeleton["Mean flat oil boiler"].fillna(populated_skeleton["Mean flat solid fuel boiler"], inplace=True)
    
    return populated_skeleton

def epcToLSOA(path, fn, LSOA_lookup):
    epc_df = getEPC(path, fn)
    org_EPC_number = len(epc_df)
    epc_df = setFlag(epc_df)
    epc_df = costTokWh(epc_df, df_fuels)
    epc_df = threshold(epc_df)
    epc_df = dwellingType(epc_df)
    epc_df = removeDuplicates(epc_df)


In [28]:
# def heatingSystem(df):

#     df["Heat system"] = None
#     df["MAINHEAT_DESCRIPTION"].fillna("No value", inplace=True)

#     gas_list = ["mains gas (not community)",
#                 #'mains gas - this is for backwards compatibility only and should not be used'
#                 ]
    
#     df.loc[df["MAIN_FUEL"].isin(gas_list), "Heat system"] = "Gas boiler"

#     electricity_list = ["electricity (not community)", 'electricity - this is for backwards compatibility only and should not be used',
#                        'Electricity: electricity, unspecified tariff']
    
#     df.loc[df["MAIN_FUEL"].isin(electricity_list), "Heat system"]='Resistance heating'
#     df.loc[df["MAINHEAT_DESCRIPTION"].str.contains("heat pump"),"Heat system"]="Heat pump"
    
#     oil_list = ['appliances able to use mineral oil or liquid biofuel','oil (not community)', 'oil - this is for backwards compatibility only and should not be used']
    
#     df.loc[df["MAIN_FUEL"].isin(oil_list), "Heat system"] = "Oil boiler"

#     solid_fuels = ['anthracite', 'house coal (not community)','wood logs', 'smokeless coal', 'house coal - this is for backwards compatibility only and should not be used',
#                   'bulk wood pellets', 'wood chips']

#     df.loc[df["MAIN_FUEL"].isin(solid_fuels), "Heat system"]="Solid fuel boiler"
    
#     df["Current heating costs [£]"] = df["HEATING_COST_CURRENT"]
#     df["Potential heating costs [£]"] = df["HEATING_COST_POTENTIAL"]
#     return df


# def dwellingType(df):
    
#     df["Dwelling type"]=None
#     df.loc[df["PROPERTY_TYPE"]=="Flat", "Dwelling type"]="Flat"
#     df.loc[df["PROPERTY_TYPE"]=="House", "Dwelling type"]=df.loc[df["PROPERTY_TYPE"]=="House", "BUILT_FORM"]
#     list_dwelling_type = list(df["Dwelling type"].unique())
#     list_dwelling_type = list(filter(None.__ne__, list_dwelling_type)) # Remove None from the list
#     # temp_df.replace({dwel:"Terrace" for dwel in list_dwelling_type if "Terrace" in dwel}, regex=True, inplace=True)
   
#     to_replace = {dwel:"Terraced" for dwel in list_dwelling_type if "Terrace" in dwel}
#     df.replace(to_replace, inplace=True)
#     df.replace("Semi-Detached", "Semi-detached", inplace=True)
#     list_dwelling_type = list(df["Dwelling type"].unique())
#     print(list_dwelling_type)
    
#     to_drop = df.loc[(df["Dwelling type"]=='NO DATA!'),:].index
#     df.drop(to_drop, inplace=True)
    
#     df["Key"] = df["Dwelling type"]+" "+df["Heat system"]
#     df["Key_EPC"] = df["CURRENT_ENERGY_RATING"]+" to "+df["POTENTIAL_ENERGY_RATING"]
#     df.dropna(subset=['Key'], inplace=True)
#     print("Shape:", df.shape)
    
    
#     return df

# def getEPC(path, file):
#     df = pd.read_csv(path+os.path.sep+file)

#     df.replace('Bungalow', 'House', regex=True, inplace=True)
#     df.replace('Maisonette', 'Flat', regex=True, inplace=True)
#     df["Heat consumption (kWh)"]=0
#     df["Space constrained ratio"]=0
#     df["Heat_rating_changes [%]"]=0
#     df["Heat_efficiency_changes [%]"]=0
#     print(df.shape)
#     return df

### Link the EPC certificate file with the recommendation file to associate each measure to a dwelling category

In [29]:
# lookup table used to improve consistency of the naming of the energy efficiency measures
lookup_measures_df = pd.read_excel(path_input_data +os.path.sep+"Lookup_table_missing_description_measures_WITH_FORMULAS.xlsx")
lookup_measures_dict=dict(zip(lookup_measures_df["IMPROVEMENT_DESCR_TEXT"],lookup_measures_df["IMPROVEMENT_ID_TEXT"]))

In [30]:
# import costs of energy efficiency measures
costs_measures_df = pd.read_csv(path_input_data + os.path.sep+"costs_domestic_efficiency_measures_by_dwelling.csv")
costs_measures_df.columns = costs_measures_df.columns.str.strip()
costs_measures_df.head()

,Dwelling type,Measures,key,Costs,Comment,Source
0,Detached,50 mm internal or external wall insulation,Detached 50 mm internal or external wall insul...,10325.0,EWI+IWI avg costs,DECC Domestic Retrofit Cost Assumptions Study
1,Flat,50 mm internal or external wall insulation,Flat 50 mm internal or external wall insulation,4575.0,EWI+IWI avg costs,DECC Domestic Retrofit Cost Assumptions Study
2,Semi-detached,50 mm internal or external wall insulation,Semi-detached 50 mm internal or external wall ...,8650.0,EWI+IWI avg costs,DECC Domestic Retrofit Cost Assumptions Study
3,Terraced,50 mm internal or external wall insulation,Terraced 50 mm internal or external wall insul...,5825.0,EWI+IWI avg costs,DECC Domestic Retrofit Cost Assumptions Study
4,Detached,Add additional 80 mm jacket to hot water cylinder,Detached Add additional 80 mm jacket to hot wa...,90.0,hot water cylinder insulation cost,DECC Domestic Retrofit Cost Assumptions Study


In [31]:
OA_lookup_file = r"PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_feb_2018.csv"
OA_lookup_df = pd.read_csv(path_input_data+os.path.sep+OA_lookup_file, low_memory=False)
OA_lookup_df.drop(["pcd8", "pcds", "dointr", "doterm", "usertype", "lsoa11nm", "msoa11nm","ladnmw", "FID"], axis=1, inplace=True) 
OA_lookup_df.drop_duplicates(inplace=True)

region_lookup_file = r"laregionlookup376las.xls"
region_lookup_df = pd.read_excel(path_input_data+os.path.sep+region_lookup_file, sheet_name=1, header=6, usecols=[0, 1, 3])

OA_lookup_df = pd.merge(OA_lookup_df, region_lookup_df, left_on='ladcd', right_on='la_code', how='left')

OA_lookup_df = pd.merge(OA_lookup_df, region_lookup_df, left_on='ladnm', right_on='la_name', how='left')
OA_lookup_df["la_code_x"].fillna(OA_lookup_df["la_code_y"], inplace=True)
OA_lookup_df["la_name_x"].fillna(OA_lookup_df["la_name_y"], inplace=True)
OA_lookup_df["region_name_x"].fillna(OA_lookup_df["region_name_y"], inplace=True)
OA_lookup_df.drop(["ladcd", "ladnm", "la_code_y","la_name_y", "region_name_y"], axis=1, inplace=True)
OA_lookup_df.dropna(subset=["la_name_x"], inplace=True)
OA_lookup_df["pcd7"]=OA_lookup_df["pcd7"].str.replace(' ','')
OA_lookup_df.columns = ['PCD7', 'OA11CD', 'LSOA11CD', 'MSOA11CD', 'LAD11CD', 'Local Authority', 'Region']

In [32]:
file_measures = 'recommendations.csv'
file_epc = 'certificates.csv'

list_measures = []
list_missing_measures = []

for fn in os.listdir(path_EPCs):
#     for LA in ["Swansea"]:
#         if LA in fn:
            print(fn)
            df = pd.read_csv(path_EPCs+os.path.sep+fn+os.path.sep+file_measures, encoding='utf-8-sig')
            
            epc_df = pd.read_csv(path_EPCs+os.path.sep+fn+os.path.sep+file_epc, encoding='utf-8-sig')
            
            LA_name = fn.split('-')[2:]
            LA_name = ''.join(LA_name)
            print(LA_name, fn)
            temp_path = path_EPCs+os.path.sep+fn+os.path.sep
            epc_df = getEPC(temp_path, file_epc)           
            epc_df = setFlag(epc_df)
            epc_df = costTokWh(epc_df)
            epc_df = threshold(epc_df)
            epc_df = dwellingType(epc_df)
            epc_df = removeDuplicates(epc_df)
            
            df.loc[(~df["IMPROVEMENT_DESCR_TEXT"].isna()) & (df["IMPROVEMENT_ID_TEXT"].isna()), "IMPROVEMENT_ID_TEXT"] = df["IMPROVEMENT_DESCR_TEXT"]
            df.replace({"IMPROVEMENT_ID_TEXT":lookup_measures_dict}, inplace=True)
                      
            print("length missing data:", len(df.loc[(~df["IMPROVEMENT_DESCR_TEXT"].isna()) & (df["IMPROVEMENT_ID_TEXT"].isna()), "IMPROVEMENT_DESCR_TEXT"].drop_duplicates()))

            df = pd.merge(df[["LMK_KEY", "IMPROVEMENT_ID_TEXT"]], epc_df[["LMK_KEY", "Key", "Dwelling type"]], right_on="LMK_KEY", left_on="LMK_KEY", how='left')
            df.dropna(inplace=True)
            
            measures_df = df["IMPROVEMENT_ID_TEXT"].value_counts().reset_index()
            
            to_merge_df = epc_df["Key"].value_counts().reset_index()
            to_merge_df.columns = ["Key", "Total"]

            measures_df = df.groupby(["Key", "IMPROVEMENT_ID_TEXT", "Dwelling type"]).count()["LMK_KEY"].reset_index()
            measures_df.columns = ["Key", "Measures", "Dwelling type", "Number"]
            measures_df = pd.merge(measures_df,to_merge_df, left_on="Key", right_on="Key", how='left') 

            measures_df["Key_measures"] = measures_df["Dwelling type"] + " " +measures_df["Measures"].astype(str)
            measures_df = pd.merge(measures_df, costs_measures_df[["key", "Costs"]], left_on="Key_measures", right_on="key", how="left")
#             measures_df.dropna(inplace=True)

            list_missing_measures= list_missing_measures +list(measures_df.loc[measures_df["Costs"].isna(), "Measures"].unique())
    
            measures_df["Total costs [£]"] = measures_df["Number"]*measures_df["Costs"]
            number_dwellings = measures_df[["Key", "Total"]].drop_duplicates().set_index("Key").sort_index()

            final_df = measures_df.groupby("Key").sum()[["Total costs [£]", "Number"]].sort_index()
            final_df.columns = ["Total costs [£]", "Number of measures"]
            
            final_df["Number of EPCs"] = number_dwellings["Total"].values
            final_df["Average costs [£]"] = final_df["Total costs [£]"]/final_df["Number of EPCs"]
            
            final_df.to_csv(saving_path+os.path.sep+LA_name+"_EPC_costs.csv", encoding='utf-8-sig')

domestic-E06000001-Hartlepool
Hartlepool domestic-E06000001-Hartlepool
(30681, 92)
list of fuels:  ['mains gas - this is for backwards compatibility only and should not be used'
 'mains gas (not community)' 'NO DATA!'
 'electricity - this is for backwards compatibility only and should not be used'
 'electricity (not community)'
 'To be used only when there is no heating/hot-water system'
 'Electricity: electricity, unspecified tariff' 'mains gas (community)'
 'INVALID!' 'oil (not community)' 'dual fuel - mineral + wood'
 'LPG - this is for backwards compatibility only and should not be used'
 'oil - this is for backwards compatibility only and should not be used'
 'smokeless coal' 'LPG (not community)' 'B30D (community)'
 'house coal (not community)' 'bulk wood pellets'
 'house coal - this is for backwards compatibility only and should not be used'
 'biomass (community)' 'wood logs' 'bottled LPG']
(10604, 97)
(89, 97)
['Terraced', 'Semi-detached', 'Detached', 'Flat', None]
Shape: (1479

(108, 97)
['Terraced', 'Flat', 'Semi-detached', 'Detached']
Shape: (26532, 100)
length missing data: 0
domestic-E06000009-Blackpool
Blackpool domestic-E06000009-Blackpool
(56641, 92)
list of fuels:  ['mains gas - this is for backwards compatibility only and should not be used'
 'electricity - this is for backwards compatibility only and should not be used'
 'mains gas (not community)' 'electricity (not community)'
 'To be used only when there is no heating/hot-water system' 'INVALID!'
 'house coal (not community)' 'NO DATA!' 'mains gas (community)'
 'oil - this is for backwards compatibility only and should not be used'
 'biomass (community)' 'smokeless coal' 'dual fuel - mineral + wood'
 'Electricity: electricity, unspecified tariff' 'LPG (not community)'
 'coal (community)' 'oil (not community)'
 'house coal - this is for backwards compatibility only and should not be used'
 'LPG - this is for backwards compatibility only and should not be used'
 'wood logs' 'bottled LPG' 'LPG specia

(25236, 97)
(189, 97)
['Terraced', 'Semi-detached', 'Detached', 'Flat', None]
Shape: (33647, 100)
length missing data: 0
domestic-E06000016-Leicester
Leicester domestic-E06000016-Leicester
(90903, 92)
list of fuels:  ['mains gas - this is for backwards compatibility only and should not be used'
 'electricity - this is for backwards compatibility only and should not be used'
 'mains gas (not community)'
 'Electricity: electricity, unspecified tariff'
 'electricity (not community)' 'NO DATA!' 'mains gas (community)'
 'smokeless coal' 'INVALID!'
 'LPG - this is for backwards compatibility only and should not be used'
 'To be used only when there is no heating/hot-water system'
 'house coal (not community)' 'LPG (not community)' 'anthracite'
 'electricity (community)' 'wood logs' 'dual fuel - mineral + wood'
 'oil - this is for backwards compatibility only and should not be used'
 'oil (not community)' 'biomass (community)'
 'house coal - this is for backwards compatibility only and should

(80, 97)
['Terraced', 'Detached', 'Flat', 'Semi-detached', None]
Shape: (20532, 100)
length missing data: 0
domestic-E06000023-Bristol
Bristol domestic-E06000023-Bristol
(125596, 92)
list of fuels:  ['mains gas (not community)' 'electricity (not community)'
 'mains gas - this is for backwards compatibility only and should not be used'
 'Electricity: electricity, unspecified tariff'
 'electricity - this is for backwards compatibility only and should not be used'
 'NO DATA!' 'mains gas (community)' 'smokeless coal' 'INVALID!'
 'To be used only when there is no heating/hot-water system' 'anthracite'
 'biomass (community)'
 'oil - this is for backwards compatibility only and should not be used'
 'house coal (not community)' 'LPG (not community)' 'oil (not community)'
 'dual fuel - mineral + wood'
 'LPG - this is for backwards compatibility only and should not be used'
 'electricity (community)' 'wood logs'
 'Community heating schemes: heat from boilers - biomass'
 'B30D (community)' 'bottl

Swindon domestic-E06000030-Swindon
(57021, 92)
list of fuels:  ['mains gas - this is for backwards compatibility only and should not be used'
 'mains gas (not community)'
 'electricity - this is for backwards compatibility only and should not be used'
 'mains gas (community)' 'Electricity: electricity, unspecified tariff'
 'INVALID!' 'smokeless coal' 'electricity (not community)'
 'oil (not community)'
 'oil - this is for backwards compatibility only and should not be used'
 'To be used only when there is no heating/hot-water system' 'NO DATA!'
 'LPG (not community)' 'house coal (not community)' 'bottled LPG'
 'LPG - this is for backwards compatibility only and should not be used'
 'house coal - this is for backwards compatibility only and should not be used'
 'electricity (community)' 'dual fuel - mineral + wood' 'wood logs'
 'anthracite' 'LPG (community)' 'bulk wood pellets']
(23597, 97)
(24, 97)
['Semi-detached', 'Terraced', 'Detached', 'Flat']
Shape: (22597, 100)
length missing dat

...

C:\Users\sceac10\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Westminster domestic-E09000033-Westminster
(87798, 92)
list of fuels:  ['mains gas - this is for backwards compatibility only and should not be used'
 'mains gas (not community)'
 'electricity - this is for backwards compatibility only and should not be used'
 'NO DATA!' 'Electricity: electricity, unspecified tariff'
 'mains gas (community)' 'electricity (not community)'
 'oil - this is for backwards compatibility only and should not be used'
 'smokeless coal'
 'To be used only when there is no heating/hot-water system' 'INVALID!'
 'LPG - this is for backwards compatibility only and should not be used'
 'electricity (community)' 'oil (community)' 'bottled LPG'
 'oil (not community)' 'LPG (community)' 'dual fuel - mineral + wood'
 'LPG (not community)'
 'house coal - this is for backwards compatibility only and should not be used'
 'B30D (community)'
 'biomass - this is for backwards compatibility only and should not be used'
 'B30K (not community)' 'biogas (community)'
 'appliances abl

(10589, 97)
(52, 97)
['Terraced', 'Detached', 'Semi-detached', 'Flat']
Shape: (13872, 100)
length missing data: 0
domestic-W06000008-Ceredigion
Ceredigion domestic-W06000008-Ceredigion
(19546, 92)
list of fuels:  ['oil - this is for backwards compatibility only and should not be used'
 'electricity - this is for backwards compatibility only and should not be used'
 'INVALID!' 'NO DATA!' 'mains gas (not community)'
 'mains gas - this is for backwards compatibility only and should not be used'
 'Electricity: electricity, unspecified tariff'
 'electricity (not community)' 'oil (not community)'
 'LPG - this is for backwards compatibility only and should not be used'
 'mains gas (community)' 'LPG (not community)' 'oil (community)'
 'dual fuel - mineral + wood' 'smokeless coal' 'bottled LPG' 'anthracite'
 'wood logs' 'To be used only when there is no heating/hot-water system'
 'house coal (not community)' 'bulk wood pellets'
 'house coal - this is for backwards compatibility only and should 

(11044, 97)
(26, 97)
['Detached', 'Terraced', 'Semi-detached', 'Flat', None]
Shape: (16218, 100)
length missing data: 0
domestic-W06000015-Cardiff
Cardiff domestic-W06000015-Cardiff
(101621, 92)
list of fuels:  ['mains gas - this is for backwards compatibility only and should not be used'
 'electricity (not community)'
 'electricity - this is for backwards compatibility only and should not be used'
 'mains gas (not community)' 'NO DATA!'
 'Electricity: electricity, unspecified tariff'
 'To be used only when there is no heating/hot-water system'
 'oil (not community)' 'INVALID!' 'smokeless coal'
 'dual fuel - mineral + wood' 'mains gas (community)'
 'oil - this is for backwards compatibility only and should not be used'
 'LPG (not community)'
 'house coal - this is for backwards compatibility only and should not be used'
 'wood logs'
 'LPG - this is for backwards compatibility only and should not be used'
 'house coal (not community)' 'bulk wood pellets' 'wood chips'
 'electricity (comm

length missing data: 0
domestic-W06000023-Powys
Powys domestic-W06000023-Powys
(34363, 92)
list of fuels:  ['electricity - this is for backwards compatibility only and should not be used'
 'mains gas (not community)'
 'mains gas - this is for backwards compatibility only and should not be used'
 'electricity (not community)' 'oil (not community)'
 'oil - this is for backwards compatibility only and should not be used'
 'LPG - this is for backwards compatibility only and should not be used'
 'bottled LPG' 'INVALID!' 'dual fuel - mineral + wood'
 'Electricity: electricity, unspecified tariff' 'LPG (not community)'
 'smokeless coal' 'oil (community)' 'wood logs'
 'To be used only when there is no heating/hot-water system'
 'mains gas (community)' 'NO DATA!' 'anthracite'
 'house coal (not community)'
 'house coal - this is for backwards compatibility only and should not be used'
 'biomass - this is for backwards compatibility only and should not be used'
 'bulk wood pellets' 'wood chips' '

In [33]:
final_df.sum()

Total costs [£]       44411.5
Number of measures       62.0
Number of EPCs           15.0
Average costs [£]     19671.5
dtype: float64